# Minimal Pipeline Notebook

Use only the cells labeled **PIPELINE** below. Legacy cells from earlier experiments are kept for reference but are not needed.

In [ ]:
# =========================
# ONE-CELL COLAB SETUP
# =========================

# --------- INSTALLS ---------
!pip install -q torch torchvision matplotlib seaborn
!git clone -q https://github.com/CSAILVision/NetDissect.git


In [1]:

# --------- IMPORTS ---------
import os, random, torch, torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import Places365
from torch.utils.data import DataLoader, Subset


In [2]:

# --------- DEVICE ---------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [3]:

# --------- DOWNLOAD PLACES365 (SAFE VERSION) ---------
os.makedirs("/content/places365", exist_ok=True)

# Validation set (RECOMMENDED – small & safe)
if not os.path.exists("/content/places365/val"):
    !wget -q http://data.csail.mit.edu/places/places365/val_256.tar
    !tar -xf val_256.tar -C /content/places365
    print("Places365 validation downloaded")

# ❗ OPTIONAL: Training data (≈24GB, COMMENTED for safety)
# Uncomment ONLY if you have enough disk space
"""
!wget http://data.csail.mit.edu/places/places365/train_256_places365standard.tar
!tar -xf train_256_places365standard.tar -C /content/places365
"""


Places365 validation downloaded


'\n!wget http://data.csail.mit.edu/places/places365/train_256_places365standard.tar\n!tar -xf train_256_places365standard.tar -C /content/places365\n'

In [6]:
# ---- FIX: Download required Places365 metadata ----
import os

os.makedirs("/content/places365", exist_ok=True)

!wget -q http://data.csail.mit.edu/places/places365/categories_places365.txt -P /content/places365
!wget -q http://data.csail.mit.edu/places/places365/filelist_places365-standard.tar -P /content/places365
!tar -xf /content/places365/filelist_places365-standard.tar -C /content/places365

print("Places365 metadata downloaded")


Places365 metadata downloaded


In [7]:

# --------- TRANSFORMS ---------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [8]:
# --------- DATASETS (COLAB-SAFE OPTION) ---------

# Use validation set for both training and validation
train_dataset = Places365(
    root="/content/places365",
    split="val",
    small=True,
    download=False,
    transform=transform
)

val_dataset = Places365(
    root="/content/places365",
    split="val",
    small=True,
    download=False,
    transform=transform
)




In [9]:
# Subset for faster training
subset_size = min(20000, len(train_dataset))
indices = random.sample(range(len(train_dataset)), subset_size)
train_subset = Subset(train_dataset, indices)

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"Train samples: {len(train_subset)}")
print(f"Val samples: {len(val_dataset)}")

Train samples: 20000
Val samples: 36500


In [10]:

# --------- LOAD PRETRAINED MODEL ---------
model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 365)  # Places365 classes
model = model.to(device)

print("Model loaded: ResNet-18 (ImageNet → Places365)")


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 197MB/s]


Model loaded: ResNet-18 (ImageNet → Places365)


In [11]:

# --------- OPTIMIZER & LOSS ---------
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# --------- TRAIN & SAVE CHECKPOINTS ---------
num_epochs = 5   # start small; increase later

val_acc_log = {}

def evaluate(model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

for epoch in range(num_epochs):
    model.train()
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = criterion(model(imgs), labels)
        loss.backward()
        optimizer.step()

    val_acc = evaluate(model)
    val_acc_log[epoch] = val_acc

    torch.save(model.state_dict(), f"/content/checkpoint_epoch_{epoch}.pth")
    print(f"Epoch {epoch} | Val Acc: {val_acc:.4f} | Checkpoint saved")

print("Training complete.")
print("Validation accuracy log:", val_acc_log)


Epoch 0 | Val Acc: 0.2450 | Checkpoint saved
Epoch 1 | Val Acc: 0.3260 | Checkpoint saved
Epoch 2 | Val Acc: 0.3658 | Checkpoint saved
Epoch 3 | Val Acc: 0.4427 | Checkpoint saved
Epoch 4 | Val Acc: 0.5081 | Checkpoint saved
Training complete.
Validation accuracy log: {0: 0.24498630136986302, 1: 0.32602739726027397, 2: 0.3658082191780822, 3: 0.4426849315068493, 4: 0.5081369863013698}


<h2>uploading the checkpoints to drive</h2>

In [12]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [13]:
import os

ckpt_dir = "/content/drive/MyDrive/semantic_mortality_checkpoints"
os.makedirs(ckpt_dir, exist_ok=True)

print("Checkpoint directory:", ckpt_dir)


Checkpoint directory: /content/drive/MyDrive/semantic_mortality_checkpoints


In [14]:
import shutil
import os

for f in os.listdir("/content"):
    if f.startswith("checkpoint_epoch_") and f.endswith(".pth"):
        src = os.path.join("/content", f)
        dst = os.path.join(ckpt_dir, f)
        shutil.copy(src, dst)

print("All checkpoints copied to Google Drive.")


All checkpoints copied to Google Drive.


In [15]:
import json

with open("/content/val_acc_log.json", "w") as f:
    json.dump(val_acc_log, f)

shutil.copy("/content/val_acc_log.json", ckpt_dir)
print("Validation logs saved.")


Validation logs saved.


<h2>how use them later </h2>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ckpt_dir = "/content/drive/MyDrive/semantic_mortality_checkpoints"
print(os.listdir(ckpt_dir))


In [ ]:
model.load_state_dict(
    torch.load(f"{ckpt_dir}/checkpoint_epoch_5.pth", map_location=device)
)


# Semantic Mortality Pipeline (Colab)

This section runs the end-to-end pipeline: training + checkpoints, NetDissect per checkpoint, tracking/analysis, and plots.

In [ ]:
# ---- PIPELINE MODE ----
# quick: small run for sanity check
# full: full Places365 training (requires train_256 download)
RUN_MODE = "quick"  # change to "full" when ready

In [ ]:
# ---- REPO SETUP ----
# Option A: mount your Drive and point to the repo
# from google.colab import drive
# drive.mount('/content/drive')
# REPO_DIR = "/content/drive/MyDrive/Semantic-Death"

# Option B: clone from GitHub
REPO_URL = "https://github.com/mstMetaly/semantic-death-4-2-thesis.git"
REPO_DIR = "/content/semantic-death-4-2-thesis"

import os
if not os.path.exists(REPO_DIR):
    !git clone -q {REPO_URL} {REPO_DIR}
else:
    !git -C {REPO_DIR} pull -q

%cd {REPO_DIR}

# Install dependencies
!pip install -q torch torchvision matplotlib seaborn scikit-image imageio

In [ ]:
# ---- PLACES365 DATASET ----
# Download Places365 (small). You can switch to full train later.
import os
os.makedirs("/content/places365", exist_ok=True)

if not os.path.exists("/content/places365/val"):
    !wget -q http://data.csail.mit.edu/places/places365/val_256.tar
    !tar -xf val_256.tar -C /content/places365

if not os.path.exists("/content/places365/categories_places365.txt"):
    !wget -q http://data.csail.mit.edu/places/places365/categories_places365.txt -P /content/places365

# Optional: download training set (large)
# !wget http://data.csail.mit.edu/places/places365/train_256_places365standard.tar
# !tar -xf train_256_places365standard.tar -C /content/places365

print("Places365 ready")

In [ ]:
# ---- PIPELINE: TRAIN + CHECKPOINT ----
import json
from pathlib import Path

CONFIG_PATH = f"{REPO_DIR}/configs/places365_resnet18.json"

cfg_path = Path(CONFIG_PATH)
if not cfg_path.exists():
    raise FileNotFoundError(f"Config not found: {cfg_path}. Make sure GitHub repo is updated and re-cloned.")

cfg = json.loads(cfg_path.read_text())
cfg["data"]["dataset_root"] = "/content/places365"
cfg["data"]["download"] = True

if RUN_MODE == "quick":
    cfg["data"]["train_split"] = "val"
    cfg["data"]["small"] = True
    cfg["data"]["max_train_samples"] = 20000
    cfg["training"]["epochs"] = 5
elif RUN_MODE == "full":
    cfg["data"]["train_split"] = "train-standard"
    cfg["data"]["small"] = False
    cfg["data"]["max_train_samples"] = None
    cfg["training"]["epochs"] = 90

cfg["netdissect"]["root"] = str(Path(REPO_DIR) / "NetDissect-Lite")

cfg_path.write_text(json.dumps(cfg, indent=2))

# Patch train.py in Colab if needed to respect download flag
train_path = Path(REPO_DIR) / "semantic_mortality" / "train.py"
if train_path.exists():
    text = train_path.read_text()
    if "download=False" in text and "download = data_cfg.get" not in text:
        text = text.replace("download=False", "download=download")
        text = text.replace("val_tf = transforms.Compose", "download = data_cfg.get(\"download\", False)\n    val_tf = transforms.Compose")
        train_path.write_text(text)

%cd {REPO_DIR}
!python -m semantic_mortality.pipeline --stage train --config {CONFIG_PATH}

In [ ]:
# ---- NETDISSECT DATA (BRODEN) ----
# Download Broden dataset required by NetDissect-Lite
%cd {REPO_DIR}/NetDissect-Lite
!bash script/dlbroden.sh
%cd {REPO_DIR}

In [ ]:
# ---- NETDISSECT PER CHECKPOINT ----
%cd {REPO_DIR}

# Ensure Broden index exists before running
import os
from pathlib import Path
broden_index = f"{REPO_DIR}/NetDissect-Lite/dataset/broden1_224/index.csv"
if not os.path.exists(broden_index):
    %cd {REPO_DIR}/NetDissect-Lite
    !bash script/dlbroden.sh
    %cd {REPO_DIR}

# Force NetDissect settings to point to the correct Broden path
settings_path = Path(f"{REPO_DIR}/NetDissect-Lite/settings.py")
if settings_path.exists():
    text = settings_path.read_text()
    text = text.replace("DATA_DIRECTORY = 'dataset/broden1_224'", f"DATA_DIRECTORY = '{REPO_DIR}/NetDissect-Lite/dataset/broden1_224'")
    text = text.replace("INDEX_FILE = 'index_sm.csv'", "INDEX_FILE = 'index.csv'")
    text = text.replace("DATASET = 'imagenet'", "DATASET = 'places365'")
    text = text.replace("NUM_CLASSES = 1000", "NUM_CLASSES = 365")
    settings_path.write_text(text)

# Patch model_loader OrderedDict bug for Colab torch build
loader_path = Path(f"{REPO_DIR}/NetDissect-Lite/loader/model_loader.py")
if loader_path.exists():
    text = loader_path.read_text()
    if "torch._C.OrderedDict" in text:
        text = text.replace("import os", "import os\nfrom collections import OrderedDict")
        text = text.replace("type(torch._C.OrderedDict())", "OrderedDict")
        loader_path.write_text(text)

# Run dissection for a single checkpoint (manual)
from pathlib import Path
import subprocess
import sys

EPOCH = 0  # set this manually

print(f"Running NetDissect for epoch {EPOCH}")
code = f"""
from pathlib import Path
from semantic_mortality.config import load_config, resolve_run_paths
from semantic_mortality.netdissect_runner import run_netdissect_per_checkpoint

cfg = load_config(Path(r\"{CONFIG_PATH}\"))
paths = resolve_run_paths(Path('runs'), cfg.run_name)
run_netdissect_per_checkpoint(cfg, paths, epochs=[{EPOCH}])
"""
# Stream output so you can see progress
import os
proc = subprocess.Popen(
    [sys.executable, "-u", "-c", code],
    cwd=REPO_DIR,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    env={**os.environ, "PYTHONUNBUFFERED": "1"},
)
for line in proc.stdout:
    print(line, end="")
ret = proc.wait()
if ret != 0:
    raise RuntimeError("NetDissect subprocess failed")

In [ ]:
# ---- NETDISSECT SINGLE EPOCH ----
# Set which epoch you want to run manually
EPOCH = 0

from pathlib import Path
from semantic_mortality.config import load_config, resolve_run_paths
from semantic_mortality.netdissect_runner import run_netdissect_per_checkpoint

cfg = load_config(Path(CONFIG_PATH))
paths = resolve_run_paths(Path("runs"), cfg.run_name)
run_netdissect_per_checkpoint(cfg, paths, epochs=[EPOCH])

In [ ]:
# ---- SAVE RUNS TO DRIVE ----
# Mount Drive and copy runs/ so results persist across sessions
from google.colab import drive
import os
import shutil
from datetime import datetime

drive.mount('/content/drive')

RUNS_DIR = f"{REPO_DIR}/runs"
STAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
DEST_DIR = f"/content/drive/MyDrive/semantic_mortality_runs_{STAMP}"

if os.path.exists(RUNS_DIR):
    shutil.copytree(RUNS_DIR, DEST_DIR, dirs_exist_ok=True)
    print("Saved runs to:", DEST_DIR)
else:
    print("runs/ not found. Make sure training/dissection finished.")

In [ ]:
# ---- DOWNLOAD RUNS (LOCAL) ----
# Zip runs/ and download to your local machine
import os
import shutil
from datetime import datetime

RUNS_DIR = f"{REPO_DIR}/runs"
STAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
ARCHIVE_BASE = f"/content/runs_{STAMP}"
ARCHIVE_PATH = shutil.make_archive(ARCHIVE_BASE, 'zip', RUNS_DIR)

print("Created:", ARCHIVE_PATH)

from google.colab import files
files.download(ARCHIVE_PATH)

In [ ]:
# ---- TRACKING + ANALYSIS + PLOTS ----
%cd {REPO_DIR}
!python -m semantic_mortality.pipeline --stage track --config {CONFIG_PATH}
!python -m semantic_mortality.pipeline --stage analyze --config {CONFIG_PATH}
!python -m semantic_mortality.pipeline --stage plot --config {CONFIG_PATH}

print("Done. Check runs/places365_resnet18/analysis and runs/places365_resnet18/plots")

In [ ]:
# ---- SANITY CHECK: MORTALITY FROM EPOCH 0-2 ----
# Uses the small trajectories file you already have
from pathlib import Path
from semantic_mortality.mortality import detect_semantic_mortality, write_events

trajectories_path = Path("ND-ON-SD/results/trajectories_epoch_0-2.csv")
output_path = Path("ND-ON-SD/results/mortality_events_epoch_0-2.csv")

if not trajectories_path.exists():
    raise FileNotFoundError(f"Missing: {trajectories_path}")

# Use a low k since we only have 3 epochs here
events = detect_semantic_mortality(
    trajectories_path=trajectories_path,
    tau=0.04,
    k=1,
    min_alive_epochs=1,
    smoothing_window=1,
)

write_events(events, output_path)
print("Events found:", len(events))
print("Saved:", output_path)

In [ ]:
# ---- COUNT INTERPRETABLE UNITS PER EPOCH ----
# Counts rows in tally.csv with score >= tau for each epoch
import csv
from pathlib import Path

TAU = 0.04
RUN_DIR = Path(f"{REPO_DIR}/runs/places365_resnet18/dissection")

if not RUN_DIR.exists():
    raise FileNotFoundError(f"Missing: {RUN_DIR}")

def count_interpretable(tally_path: Path, tau: float) -> int:
    count = 0
    with tally_path.open("r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            if float(row.get("score", 0)) >= tau:
                count += 1
    return count

results = []
for epoch_dir in sorted(RUN_DIR.glob("epoch_*")):
    tally_path = epoch_dir / "tally.csv"
    if not tally_path.exists():
        # Some runs store tally under layer subfolder
        layer_tally = epoch_dir / "layer4" / "tally.csv"
        if layer_tally.exists():
            tally_path = layer_tally
        else:
            continue
    epoch = int(epoch_dir.name.split("_")[-1])
    results.append((epoch, count_interpretable(tally_path, TAU)))

results.sort()
print("Epoch -> interpretable units (score >=", TAU, ")")
for epoch, count in results:
    print(epoch, "->", count)

In [ ]:
# ---- PIPELINE: NETDISSECT DATA (BRODEN) ----
%cd {REPO_DIR}/NetDissect-Lite
!bash script/dlbroden.sh

# Quick sanity check
import os
broden_index = "dataset/broden1_224/index.csv"
print("Broden index exists:", os.path.exists(broden_index))

%cd {REPO_DIR}

In [ ]:
# ---- PIPELINE: NETDISSECT PER CHECKPOINT ----
%cd {REPO_DIR}

# Patch netdissect runner to force DATA_DIRECTORY + INDEX_FILE
from pathlib import Path
runner_path = Path("semantic_mortality/netdissect_runner.py")
if runner_path.exists():
    text = runner_path.read_text()
    if "DATA_DIRECTORY" not in text or "INDEX_FILE" not in text:
        text = text.replace(
            "def _write_run_settings(\n    settings_path: Path,\n    *,\n    model_file: Path,\n    output_folder: Path,\n    dataset: str,\n) -> None:",
            "def _write_run_settings(\n    settings_path: Path,\n    *,\n    model_file: Path,\n    output_folder: Path,\n    dataset: str,\n    data_directory: Path,\n    index_file: str,\n) -> None:"
        )
        text = text.replace(
            "f\"DATASET = \\\"{dataset}\\\"\\n\"\n", 
            "f\"DATASET = \\\"{dataset}\\\"\\n\"\n        f\"DATA_DIRECTORY = r\\\"{data_directory.as_posix()}\\\"\\n\"\n        f\"INDEX_FILE = \\\"{index_file}\\\"\\n\"\n"
        )
        text = text.replace(
            "_write_run_settings(\n            run_settings,\n            model_file=ckpt_path,\n            output_folder=output_dir,\n            dataset=\"places365\",\n        )",
            "broden_dir = net_root / \"dataset\" / \"broden1_224\"\n        _write_run_settings(\n            run_settings,\n            model_file=ckpt_path,\n            output_folder=output_dir,\n            dataset=\"places365\",\n            data_directory=broden_dir,\n            index_file=\"index.csv\",\n        )"
        )
        runner_path.write_text(text)

!python -m semantic_mortality.pipeline --stage dissect --config {CONFIG_PATH}

In [ ]:
# ---- PIPELINE: TRACKING + ANALYSIS + PLOTS ----
%cd {REPO_DIR}
!python -m semantic_mortality.pipeline --stage track --config {CONFIG_PATH}
!python -m semantic_mortality.pipeline --stage analyze --config {CONFIG_PATH}
!python -m semantic_mortality.pipeline --stage plot --config {CONFIG_PATH}

print("Done. Check runs/places365_resnet18/analysis and runs/places365_resnet18/plots")